In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from PIL import Image
import sys

from glob import glob

np.set_printoptions(threshold=sys.maxsize)
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Links and Information Usefull:

* Documentation about TFRecords: https://www.tensorflow.org/tutorials/load_data/tfrecord
* Scripts to transform Dataset with images to TFRecords: https://keras.io/examples/keras_recipes/creating_tfrecords/

# Load Dataset

In [2]:
DATA = 'SceneNet'
MODEL = 'Real'
OUTPUT = 'output'
half = True

In [3]:
if half:
    paths = {
        'MODEL': os.path.join('/home/david/SemesterProject/Models',MODEL),
        'CKPT' : os.path.join('/home/david/SemesterProject/Models',MODEL,'CKPT'),
        'TFLITE' : os.path.join('/home/david/SemesterProject/Models','TFLITE'),
        'DATA' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half'),
        'IMAGE' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half','images'),
        'CLASS_ANN' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half','class_annotation'),
        'INSTANCE_ANN' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half','instance_annotation'),
        'COCO' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half','coco_data'),
    }
else:
    paths = {
        'MODEL': os.path.join('/home/david/SemesterProject/Models',MODEL),
        'CKPT' : os.path.join('/home/david/SemesterProject/Models',MODEL,'CKPT'),
        'TFLITE' : os.path.join('/home/david/SemesterProject/Models','TFLITE'),
        'DATA' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half'),
        'IMAGE' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half','images'),
        'CLASS_ANN' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half','class_annotation'),
        'INSTANCE_ANN' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half','instance_annotation'),
        'COCO' : os.path.join('/home/david/BlenderProc',DATA,OUTPUT,'half','coco_data'),
    }

files = {
    'OUTPUT_TFLITE_MODEL': os.path.join(paths['TFLITE'],MODEL + '.tflite'),
    'OUTPUT_TFLITE_MODEL_METADATA': os.path.join(paths['TFLITE'],MODEL + '_metadata.tflite'),
    'OUTPUT_TFLITE_LABEL_MAP': os.path.join(paths['TFLITE'],MODEL + '_tflite_label_map.txt'),
    'CREATE_COCO_TF_RECORD': os.path.join(paths['MODEL'], 'research', 'object_detection', 'dataset_tools', 'create_coco_tf_record.py'),
    'COCO_ANN': os.path.join(paths['COCO'], 'coco_annotations.json'),
}

for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

In [4]:
image_count = len(glob(os.path.join(paths["IMAGE"],"*.png")))
print("The Training Dataset contains {IMAGES_SIZE} images.".format(IMAGES_SIZE = image_count))


BATCH_SIZE = 15 #reel: einiges grösser exp 32 aber schaue bei image segmentation nach

IMG_SIZE_HEIGHT = 1280
IMG_SIZE_WIDTH = 720

NUM_CLASSES = 2



The Training Dataset contains 1 images.


In [25]:
def parse_image(img_path):
    """Load an image and its annotation (mask) and returning
    a dictionary.

    Parameters
    ----------
    img_path : str
        Image (not the mask) location.

    Returns
    -------
    dict
        Dictionary mapping an image and its annotation.
    """
    image = tf.io.read_file(img_path)
    image = tf.io.decode_png(image, channels=3)

    mask_path = tf.strings.regex_replace(img_path, "images", "class_annotation")
    mask = tf.io.read_file(mask_path)
    # The masks contain a class index for each pixels
    mask = tf.io.decode_png(mask, channels=1)

    return (image,mask)


In [43]:
dataset = tf.data.Dataset.list_files(os.path.join(paths["IMAGE"],"*.png"))
dataset = dataset.map(parse_image,num_parallel_calls=AUTOTUNE)
dataset

for im,lab in dataset:
    print(im[0:10,0:10,0])

tf.Tensor(
[[60 56 63 66 65 67 68 69 69 70]
 [50 66 67 67 67 69 68 68 68 68]
 [63 63 64 67 69 68 68 69 69 69]
 [65 66 66 67 68 67 67 67 67 67]
 [68 68 67 67 67 68 67 68 67 68]
 [68 68 67 68 68 68 68 69 68 69]
 [67 68 68 68 68 68 69 68 68 69]
 [67 67 68 68 68 68 68 69 69 69]
 [69 68 69 69 68 69 69 68 69 69]
 [70 70 69 69 69 68 68 69 69 69]], shape=(10, 10), dtype=uint8)


# Write Dataset as TFRecords

In [27]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )

In [31]:
def serialize_example(f0,f1):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'image': _image_feature(f0),
      'mask': _image_feature(f1),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

def tf_serialize_example(f0,f1):
  tf_string = tf.py_function(
    serialize_example,
    (f0,f1),  # Pass these args to the above function.
    tf.string)      # The return type is `tf.string`.
  return tf.reshape(tf_string, ()) # The result is a scalar.



In [32]:
dataset = tf.data.Dataset.list_files(os.path.join(paths["IMAGE"],"*.png"),seed = SEED)
dataset = dataset.map(parse_image,num_parallel_calls=AUTOTUNE)
serialized_dataset = dataset.map(tf_serialize_example)
filename = 'test.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_dataset)

# Read TFRecords Dataset

In [34]:
def parse_tfrecord_fn(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "mask": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
    example["mask"] = tf.io.decode_jpeg(example["mask"], channels=1)
    return example

In [38]:
filenames = ['test.tfrecord']
dataset = tf.data.TFRecordDataset(filenames)

dataset = dataset.map(parse_tfrecord_fn)
dataset

<MapDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'mask': TensorSpec(shape=(None, None, 1), dtype=tf.uint8, name=None)}>

In [42]:
for im,lab in dataset:
    print(im[0:10,0:10,0])

tf.Tensor(
[[60 56 63 66 65 67 68 69 69 70]
 [50 66 67 67 67 69 68 68 68 68]
 [63 63 64 67 69 68 68 69 69 69]
 [65 66 66 67 68 67 67 67 67 67]
 [68 68 67 67 67 68 67 68 67 68]
 [68 68 67 68 68 68 68 69 68 69]
 [67 68 68 68 68 68 69 68 68 69]
 [67 67 68 68 68 68 68 69 69 69]
 [69 68 69 69 68 69 69 68 69 69]
 [70 70 69 69 69 68 68 69 69 69]], shape=(10, 10), dtype=uint8)
